# Data Cleaning and Geocoding for Housing Dataset

I will be working with a dataset called `PPR-2023.csv` containing the prices for houses listed in Ireland in 2023. I obtained the dataset from https://www.propertypriceregister.ie/website/npsra/pprweb.nsf/PPRDownloads?OpenForm&File=PPR-2023.csv&County=ALL&Year=2023&Month=ALL 

The goal is to clean the dataset and perform geocoding to get the longitude and latitude of each address which will be used for creating visualisation of a map to show the distribution of housing and their prices in Ireland.

I will also be cleaning the `DEA06.csv` dataset which is the Earnings Distribution.


## Data Cleaning

In [10]:
import pandas as pd
import requests
from tqdm import tqdm

#load the dataset
df = pd.read_csv('PPR-2023.csv')

#drop unnecessary columns
df.drop(columns='Date of Sale (dd/mm/yyyy)',inplace=True)
df.drop(columns='Property Size Description',inplace=True)
df.drop(columns='Eircode',inplace=True)

In [11]:
#clean the Price column by removing symbols and convert the value to float
df.rename(columns={'Price (�)': "Price"},inplace=True)

df['Price'] = df['Price'].str.replace('�', '', regex=True)  # Remove currency symbol
df['Price'] = df['Price'].str.replace(',', '', regex=True)  # Remove commas
df['Price'] = df['Price'].astype(float)  # Convert to float

In [12]:
#clean the Address column to make it easier for geocoding
def clean_address(address):
    # Convert to Title Case
    address = address.title()
    
    # Remove extra spaces
    address = " ".join(address.split())
    
    # Standardize common terms
    address = address.replace(" Rd", " Road").replace(" St", " Street")
    address = address.replace(" Ave", " Avenue").replace(" Co.", " County")
    address = address.replace(" Dublin ", " Dublin, ")
    address = address.replace(" ,", ",")  
    address = address.replace(", Dublin,", ", Dublin") 
    
    return address

df['Address'] = df['Address'].apply(clean_address)

## Geocoding

In this section, I will make an API to get the latitude and longitude for each address in the dataset. I will use a platform called Mapbox to do this.

In [13]:
#insert token
MAPBOX_ACCESS_TOKEN = "token"

MAPBOX_GEOCODING_URL = "https://api.mapbox.com/geocoding/v5/mapbox.places"


In [ ]:
def get_coordinates(address):
    """
    Use Mapbox API to fetch the latitude and longitude for an address.
    
    Parameters:
        address (str): Address to geocode.
    
    Returns:
        tuple: (latitude, longitude) or (None, None) if not found.
    """
    try:
        url = f"{MAPBOX_GEOCODING_URL}/{address}.json"
        params = {
            "access_token": MAPBOX_ACCESS_TOKEN,
            "limit": 1  # Return only the best match
        }
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an error for HTTP issues
        data = response.json()
        
        if "features" in data and data["features"]:
            coords = data["features"][0]["geometry"]["coordinates"]
            return coords[1], coords[0]  # (lat, lon)
        else:
            return None, None
    except Exception as e:
        print(f"Error fetching coordinates for {address}: {e}")
        return None, None
    
tqdm.pandas(desc="Geocoding Addresses")
df["Latitude"], df["Longitude"] = zip(*df["Address"].progress_apply(get_coordinates))

However I noticed that some of the latitude and longitude that the code return is not correct. So I need to ensure that the latitude and longitude returned fall within Ireland's geographical bounds.

In [ ]:
#Define Ireland's bounding box
IRELAND_BOUNDS = {
    "min_lat": 51.4,
    "max_lat": 55.4,
    "min_lon": -10.5,
    "max_lon": -5.5
}

def is_in_ireland(lat, lon):
    """
    Check if the latitude and longitude fall within Ireland's bounding box.
    """
    if lat is None or lon is None:
        return False
    return (
        IRELAND_BOUNDS["min_lat"] <= lat <= IRELAND_BOUNDS["max_lat"] and
        IRELAND_BOUNDS["min_lon"] <= lon <= IRELAND_BOUNDS["max_lon"]
    )

df["Valid_Geocode"] = df.apply(lambda row: is_in_ireland(row["Latitude"], row["Longitude"]), axis=1)

#Filter rows with valid geocodes
df_valid = df[df["Valid_Geocode"]]

#save the valid rows
df_valid.to_csv("valid_geocodes.csv", index=False)

## Data Cleaning for Earnings Distribution

In [ ]:
#load the dataset
df2 = pd.read_csv('DEA06.csv')

#drop unnecessary columns
df2.drop(columns='Statistic Label',inplace=True)
df2.drop(columns='Year',inplace=True)
df2.drop(columns='Unit',inplace=True)

In [ ]:
#CHATGPT
county_coords = {
    'County': ['Carlow', 'Cavan', 'Clare', 'Cork', 'Donegal', 'Dublin', 'Galway', 'Kerry', 
               'Kildare', 'Kilkenny', 'Laois', 'Leitrim', 'Limerick', 'Longford', 'Louth', 
               'Mayo', 'Meath', 'Monaghan', 'Offaly', 'Roscommon', 'Sligo', 'Tipperary', 
               'Waterford', 'Westmeath', 'Wexford', 'Wicklow'],
    'Latitude': [52.8365, 53.9908, 52.8470, 51.8985, 54.6547, 53.3498, 53.2707, 52.1545, 
                 53.1624, 52.6541, 53.0336, 54.1000, 52.6638, 53.7271, 53.9128, 53.9111, 
                 53.6052, 54.2496, 53.2734, 53.7596, 54.2697, 52.6793, 52.2593, 53.5257, 
                 52.3369, 52.9862],
    'Longitude': [-6.9341, -7.3602, -8.9885, -8.4756, -8.1041, -6.2603, -9.0568, -9.5669, 
                  -6.9130, -7.2525, -7.3003, -8.0000, -8.6267, -7.7980, -6.5400, -9.0500, 
                  -6.6550, -6.9684, -7.4900, -8.1960, -8.4694, -8.1612, -7.1101, -7.3380, 
                  -6.4620, -6.3673]
}

county_df = pd.DataFrame(county_coords)

df2 = df2.merge(county_df, on='County', how='left')

df2.to_csv("earnings_dist.csv", index=False)